In [1]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='ssenigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [ ]:
conf = SparkConf().setAppName('Join_with_Using').setMaster('yarn') 
spark = SparkSession.builder.config(conf=conf).getOrCreate()

<!-- # original:
+----+---+-----+
|year|qrt|  num|
+----+---+-----+
|2022|  2|    3|
|2022|  3|    5|
|2022|  3|    1|
|2021|  3|    3|
|2021|  1|    2|
+----+---+-----+

# pivoted:
+----+------------+------------+------------+------------+
|year|qrt1_sum_num|qrt2_sum_num|qrt3_sum_num|qrt4_sum_num|
+----+------------+------------+------------+------------+
|2022|        null|           3|           6|        null|
|2021|           2|        null|           3|        null|
+----+------------+------------+------------+------------+ -->

In [36]:
from pyspark.sql.functions import rand, lit
from pyspark.sql.types import IntegerType

# create sample DataFrame with quartal data
df = spark.range(1000)
df = df.withColumn('quarter', (4*rand()).cast(IntegerType())%4+1)
df = df.withColumn('year', lit(2025))
# create table to address it in sql
df.createOrReplaceTempView("t_quarter_data") 

# create sample DataFrame with quartals names
sql = """
   select * from values 
    (2025, 1, 'Q1 2025'), (2025, 2, 'Q2 2025'), 
    (2025, 3, 'Q3 2025'), (2025, 4, 'Q4 2025') 
             as (year, quarter, quarter_name) """
spark.sql(sql).createOrReplaceTempView("t_quarter_names")

In [48]:
sql = """ 
  select 
    t_quarter_names.quarter_name, count(1) cnt
  from t_quarter_data join t_quarter_names
        using (year, quarter) 
  group by t_quarter_names.quarter_name
  order by t_quarter_names.quarter_name """

spark.sql(sql).show(truncate=False)

+------------+---+
|quarter_name|cnt|
+------------+---+
|Q1 2025     |272|
|Q2 2025     |243|
|Q3 2025     |242|
|Q4 2025     |243|
+------------+---+



In [49]:
sql = """ 
  select 
    t_quarter_names.quarter_name, count(1) cnt
  from t_quarter_data join t_quarter_names
        using (quarter_name) 
  group by t_quarter_names.quarter_name
  order by t_quarter_names.quarter_name """

spark.sql(sql).show(truncate=False)
# AnalysisException: [UNRESOLVED_USING_COLUMN_FOR_JOIN] 
# USING column `quarter_name` cannot be resolved 
# on the left side of the join. 
# The left-side columns: [`id`, `quarter`, `year`].

AnalysisException: [UNRESOLVED_USING_COLUMN_FOR_JOIN] USING column `quarter_name` cannot be resolved on the left side of the join. The left-side columns: [`id`, `quarter`, `year`].

In [50]:
spark.stop()